In [3]:
import datacube
import numpy as np

import gdal
gdal.UseExceptions()
from typing import List


def array_to_geotiff_multiband(fname: str, data: List[np.ndarray],
                               geo_transform: tuple, projection: str,
                               nodata_val=0, dtype=gdal.GDT_Float32):
    """ Create a multiband GeoTIFF file with data from an array.
    fname : output geotiff file path including extension
    data : list of numpy arrays
    geo_transform : Geotransform for output raster; e.g.
    "(upleft_x, x_size, x_rotation, upleft_y, y_rotation, y_size)"
    projection : WKT projection for output raster
    nodata_val : Value to convert to nodata in the output raster; default 0
    dtype : gdal dtype object, optional
        Optionally set the dtype of the output raster; can be
        useful when exporting an array of float or integer values. """
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data[0].shape  # Create raster of given size and projection
    dataset = driver.Create(fname, cols, rows, len(data), dtype)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    for idx, d in enumerate(data):
        band = dataset.GetRasterBand(idx + 1)
        band.WriteArray(d)
        band.SetNoDataValue(nodata_val)
    dataset = None  # Close %%file

In [5]:
dc = datacube.Datacube(app="s2-l1c-eu")
sets = dc.find_datasets(product="s2a_level1c_granule")
bands = ['B02', 'B03', 'B04']  # rgb

In [6]:
for idx, dataset in enumerate(sets):
    print(idx, dataset.metadata_doc["properties"]["cloudy_pixel_percentage"])

0 18.3903
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 2.5419
8 0.0
9 0.0
10 2.5041
11 0.0
12 0.0
13 1.0532
14 2.2702
15 1.3851
16 17.5413
17 0.0
18 98.8057
19 97.0854
20 0.0394
21 1.1356
22 94.3767
23 1.3005
24 22.2817


In [7]:
dataset = sets[16]
dataset.metadata_doc["properties"]

{'tile_id': 'S2A_OPER_MSI_L1C_TL_EPAE_20201002T095127_A027574_T35PPM_N02.09',
 'datetime': '2020-10-02T08:37:32.329132Z',
 'eo:platform': 'SENTINEL-2A',
 'eo:instrument': 'MSI',
 'mean_sun_zenith': 24.0422336927737,
 'odc:file_format': 'JPEG2000',
 'odc:region_code': '35PPM',
 'mean_sun_azimuth': 125.130263338263,
 'cloudy_pixel_percentage': 17.5413}

In [10]:
offset = 40000  # offset from edges
b = dataset.bounds
ds = dc.load(product='s2a_level1c_granule',
             dask_chunks={},
             output_crs='epsg:32635',
             resolution=(-10,10),
             x=(b.left+offset, b.right-offset),
             y=(b.top-offset, b.bottom+offset),
             crs="epsg:32635",
             datasets=[dataset])
geotrans = ds.geobox.transform.to_gdal()
prj = ds.geobox.crs.wkt

In [93]:
tile_id = dataset.metadata_doc["properties"]["tile_id"]
dataset_id = dataset.id
tile_props = dataset.metadata_doc["properties"]
shadow_azimuth = 90-tile_props["mean_sun_azimuth"]
CLR_PRJ_DIST = 1  # maximum distance to search for cloud shadows from cloud edges
az = np.deg2rad(tile_props["mean_sun_azimuth"])
x = np.math.cos(az)
y = np.math.sin(az)
shift_distance = 30
x *= shift_distance
y *= shift_distance

In [94]:
# create rgb image
bands = ['B02', 'B03', 'B04']
values = [np.squeeze(ds[band].values.astype('float64')) / 10000 for band in bands]
# values = [np.squeeze(v) for v in values]
# values = [v.astype('float64') for v in values]
# values = [v / 10000 for v in values]
array_to_geotiff_multiband(f'/home/mikael/s2data/{tile_id}_rgb.tif', values, geotrans, prj)

In [12]:
arr = ds.to_array().values.astype('float64')
arr /= 10000
print(arr.shape)
arr = np.moveaxis(arr, 0, -1)  # shift to n_images, x, y, n_bands
print(arr.shape)

(13, 1, 2980, 2980)
(1, 2980, 2980, 13)


In [13]:
from s2cloudless import S2PixelCloudDetector
cloud_detector = S2PixelCloudDetector(threshold=0.3, average_over=1, dilation_size=1, all_bands=True)
cloud_masks = cloud_detector.get_cloud_masks(arr)
cloud_masks = np.squeeze(cloud_masks)
rows, cols = cloud_masks.shape

(1, 2980, 2980)
(2980, 2980)


In [84]:
new_rows = np.zeros((abs(int(y)), cols))
new_cols = np.zeros((rows, abs(int(x))))
new_rows[:] = 2
new_cols[:] = 2
print(new_cols.shape, new_rows.shape)

(2980, 17) (24, 2980)


In [85]:
cm0 = cloud_masks.copy()
print(cm0.shape)
if y > 0:
    cm1 = np.append(cm0, new_rows, axis=0)
    cm1 = cm1[int(y):, :]
    # cm1 = cm1[abs(int(y)):, :]
else:
    cm1 = np.append(new_rows, cm0, axis=0)
    cm1 = cm1[:int(y), :]
    # cm1 = cm1[:-int(y), :]
print(cm1.shape)
if x < 0:
    cm2 = np.append(new_cols, cm1, axis=1)
    cm2 = cm2[:, :int(x)]
else:
    cm2 = np.append(cm1, new_cols, axis=1)
    cm2 = cm2[:, int(x):]
print(cm2.shape)

(2980, 2980)
(2980, 2980)
(2980, 2980)


In [89]:
nir = arr[:, :, :, 7]
dark_pixels = np.squeeze(np.where(nir <= 0.15, 1, 0))

In [90]:
cloud_shadows = np.where((cloud_masks == 0) & (cm2 == 1) & (dark_pixels == 1), 1, 0)

In [82]:
# non-directional shadow masking
# from scipy import ndimage
# cld_proj = ndimage.distance_transform_edt(np.invert(cloud_masks) + 2)  # TODO: perform directional distance transform
# cld_proj = np.where((cld_proj < 40) & (cld_proj > 0), True, False)
# both = np.where(dark_pixels & cld_proj, True, False)
# nocloud = np.where(both & (cloud_masks != 1), True, False)

In [92]:
cloud_shadow_values = [cloud_shadows]
array_to_geotiff_multiband(f'/home/mikael/s2data/{tile_id}_shadows.tif', cloud_shadow_values, geotrans, prj)
cloud_mask_values = [cloud_masks]
array_to_geotiff_multiband(f'/home/mikael/s2data/{tile_id}_clouds.tif', cloud_mask_values, geotrans, prj)